In [74]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
data=pd.read_csv("event.csv")

In [76]:
data.fillna(int(0),inplace = True)
data.head(10)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,0.0
1,1433224214164,992329,view,248676,0.0
2,1433221999827,111016,view,318965,0.0
3,1433221955914,483717,view,253185,0.0
4,1433221337106,951259,view,367447,0.0
5,1433224086234,972639,view,22556,0.0
6,1433221923240,810725,view,443030,0.0
7,1433223291897,794181,view,439202,0.0
8,1433220899221,824915,view,428805,0.0
9,1433221204592,339335,view,82389,0.0


In [77]:
print data.groupby("event").groups

{'transaction': Int64Index([   130,    304,    418,    814,    843,   1214,   1215,   1234,
              1350,   1512,
            ...
            195385, 195390, 195412, 195503, 196084, 196369, 196452, 196455,
            196707, 196818],
           dtype='int64', length=1535), 'addtocart': Int64Index([    17,     19,     63,    112,    179,    302,    456,    458,
               542,    556,
            ...
            196504, 196553, 196587, 196626, 196657, 196726, 196737, 196806,
            196862, 196874],
           dtype='int64', length=4597), 'view': Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            196895, 196896, 196897, 196898, 196899, 196900, 196901, 196902,
            196903, 196904],
           dtype='int64', length=190773)}


In [78]:
data.drop(['transactionid'],axis = 1)
data.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,0.0
1,1433224214164,992329,view,248676,0.0
2,1433221999827,111016,view,318965,0.0
3,1433221955914,483717,view,253185,0.0
4,1433221337106,951259,view,367447,0.0


In [79]:
data.event = data.event.map({'view':0, 'transaction':1, 'addtocart':2})
print(data.event[0])

0


In [80]:
events_c = data.event.value_counts()
events_c.head()
items_c = data.itemid.value_counts()
items_c.head()

5411      715
194791    286
309778    281
298009    202
370653    199
Name: itemid, dtype: int64

In [81]:
data_val = data.values
print(data_val.shape)
X = data.loc[:, ['timestamp', 'visitorid', 'itemid','transactionid']]
print(X.shape)
Y = data.loc[:,['event']]
print(Y.shape)

(196905, 5)
(196905, 4)
(196905, 1)


In [82]:
X_val = X.values
Y_val = Y.values
print(X_val.shape, Y_val.shape)

((196905, 4), (196905, 1))


In [83]:
X_train, X_valid, Y_train, Y_test = train_test_split(X_val,Y_val, test_size = 0.20, random_state = 36)

In [84]:
YY=Y_test

In [85]:
y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)
print(y_train.shape)
print(y_test.shape)

(157524, 3)
(39381, 3)


In [86]:
X_train=X_train.reshape((X_train.shape[0],1,X_train.shape[1]))

In [87]:
X_valid=X_valid.reshape((X_valid.shape[0],1,X_valid.shape[1]))

In [88]:
X_train.shape

(157524, 1, 4)

In [89]:
from keras.layers import Input, LSTM, Embedding, Dense,Activation
from keras.models import Sequential

In [90]:
embed_dim = 128
lstm_out = 200
batch_size = 32

In [91]:
model = Sequential()
model.add(LSTM(30,return_sequences=True,input_shape=(1,4)))
model.add(LSTM(30,return_sequences=True))
model.add(LSTM(30))
model.add(Dense(3,activation="softmax"))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 1, 30)             4200      
_________________________________________________________________
lstm_22 (LSTM)               (None, 1, 30)             7320      
_________________________________________________________________
lstm_23 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 93        
Total params: 18,933
Trainable params: 18,933
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, batch_size =batch_size, epochs = 5,class_weight="balanced")

Epoch 1/5
157524/157524 [==============================] - 42s 264us/step - loss: 0.1664 - acc: 0.9682
Epoch 2/5
157524/157524 [==============================] - 26s 165us/step - loss: 0.1574 - acc: 0.9687
Epoch 3/5
 84896/157524 [===============>..............] - ETA: 11s - loss: 0.1591 - acc: 0.9682

In [36]:
model.evaluate(X_valid,y_test)

39381/39381 [==============================] - 4s 102us/step


[0.153839262011156, 0.9694268809832153]

In [42]:
y_pred=model.predict_classes(X_valid)

In [58]:
from sklearn.metrics import classification_report

In [60]:
print classification_report(YY,y_pred)

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     38177
           1       0.00      0.00      0.00       297
           2       0.00      0.00      0.00       907

   micro avg       0.97      0.97      0.97     39381
   macro avg       0.32      0.33      0.33     39381
weighted avg       0.94      0.97      0.95     39381

